In [1]:
import os 
os.chdir("D:\Motion-Detection-and-Tracking-Using-Opencv-main\YOLO")
import supervision as sv 
from ultralytics import YOLO
import cv2
import numpy as np

In [2]:
model=YOLO('yolov8s.pt')

In [25]:
w,h=640,480
forcc=cv2.VideoWriter_fourcc(*'XVID')
out=cv2.VideoWriter('people count in market by supervision.mp4',forcc,20.0,(w,h))
labels=model.names
cap=cv2.VideoCapture('test-video.mp4')
zones=[np.array([
[374, 159],[270, 251],[170, 223],[122, 211],[42, 203],[2, 195],[2, 163],[10, 159],[158, 175],[282, 179],[374, 159]
])
,np.array([
[378, 163],[590, 195],[582, 163],[638, 119],[638, 75],[610, 79],[478, 135],[382, 163]
]),
np.array([
[374, 167],[586, 195],[478, 343],[270, 255],[370, 167]
]),np.array([
[586, 195],[486, 343],[634, 415],[638, 247],[590, 203]
])]

polygons=[sv.PolygonZone(zone,[w,h]) for zone in zones]
colors=sv.ColorPalette.default()
polygon_annonators=[sv.PolygonZoneAnnotator(polygon,color=colors.by_idx(idx)) for idx,polygon in enumerate(polygons) ]
box_annonators=[sv.BoxAnnotator(color=colors.by_idx(idx), thickness= 2,text_scale=0.3,text_thickness= 1,text_padding=2) 
               for idx,zone in enumerate(zones)]
while cap.isOpened():
    _,frame=cap.read()
    frame=cv2.resize(frame,(w,h))
    result=model(frame)[0]
    detections=sv.Detections.from_yolov8(result)
    detections=detections[detections.class_id==0]
    
    for box_annonator,polygon_annonator,polygon in zip(box_annonators,polygon_annonators,polygons):
        mask=polygon.trigger(detections)
        test=detections[mask]
        label=[f'{labels[idx]} {conf:0.2f}' for _ ,_, conf ,idx ,_ in test]
        filter_detections=detections[mask]
        box_annonator.annotate(frame,filter_detections,label)
        polygon_annonator.annotate(frame)
        

    out.write(frame)
    cv2.imshow('frame',frame)
    if cv2.waitKey(1)==27:
        break
out.release()
cap.release()
cv2.destroyAllWindows()


0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 4361.4ms
Speed: 17.8ms preprocess, 4361.4ms inference, 9.0ms postprocess per image at shape (1, 3, 480, 640)
C:\Users\5g\AppData\Local\Temp\ipykernel_16152\2177646738.py:27: DeprecationWarning: from_yolov8 is deprecated: 
        This method is deprecated and removed in 0.15.0 release.
        Use sv.Detections.from_ultralytics() instead as it is more generic and
        can be used for detections from any ultralytics.engine.results.Results Object
        
  detections=sv.Detections.from_yolov8(result)

0: 480x640 3 persons, 1 car, 1 truck, 1 bird, 3404.0ms
Speed: 63.9ms preprocess, 3404.0ms inference, 8.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 2295.0ms
Speed: 15.0ms preprocess, 2295.0ms inference, 9.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 1954.0ms
Speed: 100.0ms preprocess, 1954.0ms inference, 11.0ms postprocess per image at shape (1, 3, 4